# Reading Data

In [1]:
import json
import re
import numpy as np
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field, asdict
from llama_index.core import Document

# Load data
with open("/home/ltnga/LawVN-Instructction-Gen/src/data/data.json") as f:
    all_data = json.load(f)

documents = all_data

ModuleNotFoundError: No module named 'llama_index'

# Custom Legal Document Processing

In [ ]:
@dataclass
class LegalReference:
    """Lưu thông tin về tham chiếu pháp lý."""
    article: Optional[str] = None
    paragraph: Optional[str] = None
    point: Optional[str] = None
    is_current_article: bool = False
    raw_text: str = ""

@dataclass
class ChunkMetadata:
    """Metadata cho chunk pháp lý."""
    article_id: str
    title: str = ""
    type: str = "article"  # article, article_part, paragraph
    paragraphs: List[str] = field(default_factory=list)
    points: List[str] = field(default_factory=list)
    references: List[LegalReference] = field(default_factory=list)

@dataclass
class LegalChunk:
    """Một đoạn văn bản pháp lý với metadata."""
    text: str
    metadata: ChunkMetadata
    embedding: Optional[np.ndarray] = None
    
    def to_dict(self):
        """Chuyển đổi thành dict tương thích với llama_index"""
        return {
            "text": self.text,
            "metadata": asdict(self.metadata),
            "embedding": self.embedding.tolist() if self.embedding is not None else None
        }

In [ ]:
class LegalDocument:
    """Xử lý tài liệu pháp lý để chunking và embedding."""
    
    def __init__(self, max_chunk_tokens=500, overlap_tokens=100):
        self.max_chunk_tokens = max_chunk_tokens
        self.overlap_tokens = overlap_tokens
        self.reference_patterns = {
            "other_article": re.compile(r'(điểm|khoản) ([a-z]|\d+)(?:,\s*(điểm|khoản) ([a-z]|\d+))* (Điều|khoản) (\d+)', re.IGNORECASE),
            "same_article": re.compile(r'(điểm|khoản) ([a-z]|\d+)(?:,\s*(điểm|khoản) ([a-z]|\d+))* (Điều này)', re.IGNORECASE),
            "article_only": re.compile(r'Điều (\d+)', re.IGNORECASE)
        }
    
    def process(self, text: str) -> List[LegalChunk]:
        """Xử lý văn bản pháp lý thành các chunk."""
        # Phân tách văn bản thành các điều
        articles = re.split(r'(Điều \d+\.)', text)
        chunks = []
        
        for i in range(1, len(articles), 2):
            article_title = articles[i].strip()
            article_content = articles[i+1].strip() if i+1 < len(articles) else ""
            
            # Trích xuất số điều
            article_id = re.search(r'Điều (\d+)\.', article_title).group(1)
            
            # Trích xuất tiêu đề điều
            title_match = re.search(r'^([^0-9\.\n]*)(?=\d+\.|$)', article_content)
            title = title_match.group(1).strip() if title_match else ""
            
            # Quyết định phương pháp phân chunk
            article_chunks = self._process_article(
                article_title, article_id, title, article_content
            )
            chunks.extend(article_chunks)
        
        return chunks
    
    def _process_article(self, article_title, article_id, title, article_content) -> List[LegalChunk]:
        """Xử lý một điều thành các chunk."""
        # Tìm các tham chiếu trong điều
        references = self._extract_references(article_content, article_id)
        
        # Đếm tokens (ước tính theo số từ)
        token_count = len(article_content.split())
        
        if token_count <= self.max_chunk_tokens:
            # Điều ngắn: giữ nguyên toàn bộ
            metadata = ChunkMetadata(
                article_id=article_id,
                title=title,
                type="article",
                references=references
            )
            
            return [LegalChunk(
                text=article_title + " " + article_content,
                metadata=metadata
            )]
        else:
            # Điều dài: phân theo khoản
            return self._split_by_paragraphs(
                article_title, article_id, title, article_content, references
            )
    
    def _split_by_paragraphs(
        self, article_title, article_id, title, article_content, references
    ) -> List[LegalChunk]:
        """Phân tách điều thành các chunk theo khoản."""
        chunks = []
        
        # Tách khoản
        paragraphs = re.split(r'(\d+\.)', article_content)
        current_chunk_text = article_title + " " + title
        current_paragraphs = []
        current_refs = []
        
        for j in range(1, len(paragraphs), 2):
            if j+1 < len(paragraphs):
                para_num = paragraphs[j].strip().replace(".", "")
                para_content = paragraphs[j+1].strip()
                
                # Tìm tham chiếu trong khoản
                para_refs = self._extract_references(para_content, article_id)
                
                # Kiểm tra độ dài sau khi thêm khoản mới
                candidate = current_chunk_text + " " + para_num + ". " + para_content
                candidate_tokens = len(candidate.split())
                
                if candidate_tokens <= self.max_chunk_tokens:
                    # Thêm khoản vào chunk hiện tại
                    current_chunk_text = candidate
                    current_paragraphs.append(para_num)
                    current_refs.extend(para_refs)
                else:
                    # Lưu chunk hiện tại
                    metadata = ChunkMetadata(
                        article_id=article_id,
                        title=title,
                        type="article_part",
                        paragraphs=current_paragraphs.copy(),
                        references=current_refs.copy()
                    )
                    
                    chunks.append(LegalChunk(
                        text=current_chunk_text,
                        metadata=metadata
                    ))
                    
                    # Bắt đầu chunk mới với bối cảnh
                    # Thêm tiêu đề điều và khoản này
                    current_chunk_text = f"{article_title} (tiếp) {para_num}. {para_content}"
                    current_paragraphs = [para_num]
                    current_refs = para_refs.copy()
        
        # Thêm chunk cuối cùng
        if current_paragraphs:
            metadata = ChunkMetadata(
                article_id=article_id,
                title=title,
                type="article_part",
                paragraphs=current_paragraphs,
                references=current_refs
            )
            
            chunks.append(LegalChunk(
                text=current_chunk_text,
                metadata=metadata
            ))
        
        return chunks
    
    def _extract_references(self, text: str, current_article_id: str) -> List[LegalReference]:
        """Trích xuất tham chiếu từ văn bản."""
        references = []
        
        # Tìm tham chiếu đến điều khác
        for match in self.reference_patterns["other_article"].finditer(text):
            ref = LegalReference(
                article=match.group(6),
                paragraph=match.group(2) if match.group(1).lower() == "khoản" else None,
                point=match.group(2) if match.group(1).lower() == "điểm" else None,
                raw_text=match.group(0)
            )
            references.append(ref)
        
        # Tìm tham chiếu trong cùng điều
        for match in self.reference_patterns["same_article"].finditer(text):
            ref = LegalReference(
                article=current_article_id,
                paragraph=match.group(2) if match.group(1).lower() == "khoản" else None,
                point=match.group(2) if match.group(1).lower() == "điểm" else None,
                is_current_article=True,
                raw_text=match.group(0)
            )
            references.append(ref)
        
        # Tìm tham chiếu chỉ đến điều
        for match in self.reference_patterns["article_only"].finditer(text):
            article_id = match.group(1)
            # Tránh trùng lặp với các điều đã tìm thấy
            if not any(r.article == article_id and r.paragraph is None and r.point is None for r in references):
                ref = LegalReference(
                    article=article_id,
                    raw_text=match.group(0)
                )
                references.append(ref)
        
        return references

In [ ]:
from sentence_transformers import SentenceTransformer

class LegalEmbedding:
    """Tạo và quản lý embedding cho tài liệu pháp lý."""
    
    def __init__(self, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
        self.model = SentenceTransformer(model_name)
    
    def create_embeddings(self, chunks: List[LegalChunk], method='enhanced') -> List[LegalChunk]:
        """Tạo embedding cho các chunk theo phương pháp chỉ định."""
        if method == 'basic':
            return self._create_basic_embeddings(chunks)
        elif method == 'enhanced':
            return self._create_enhanced_embeddings(chunks)
        elif method == 'hierarchical':
            return self._create_hierarchical_embeddings(chunks)
        else:
            raise ValueError(f"Phương pháp embedding không hợp lệ: {method}")
    
    def _create_basic_embeddings(self, chunks: List[LegalChunk]) -> List[LegalChunk]:
        """Tạo embedding cơ bản cho các chunk."""
        texts = [chunk.text for chunk in chunks]
        embeddings = self.model.encode(texts)
        
        for i, chunk in enumerate(chunks):
            chunk.embedding = embeddings[i]
        
        return chunks
    
    def _create_enhanced_embeddings(self, chunks: List[LegalChunk]) -> List[LegalChunk]:
        """Tạo embedding nâng cao với tính năng pháp lý."""
        
        for chunk in chunks:
            # 1. Tạo văn bản nâng cao với trọng số cho các phần quan trọng
            enhanced_text = chunk.text
            
            # Tăng cường tiêu đề điều và số điều
            article_title_match = re.search(r'(Điều \d+\.\s*[^\.\']+)', enhanced_text)
            if article_title_match:
                article_title = article_title_match.group(1)
                # Lặp lại tiêu đề 2 lần để tăng trọng số trong embedding
                enhanced_text = article_title + " " + article_title + " " + enhanced_text
            
            # Tăng cường số khoản, điểm
            para_points = re.findall(r'(\d+\.\s*[^\.\']+|[a-z]\)\s*[^;]+)', enhanced_text)
            if para_points:
                # Thêm các khoản, điểm vào đầu văn bản để tăng trọng số
                para_points_text = " ".join(para_points[:3])  # Giới hạn 3 khoản/điểm đầu tiên
                enhanced_text = para_points_text + " " + enhanced_text
            
            # 2. Tạo embedding cho văn bản đã tăng cường
            chunk.embedding = self.model.encode(enhanced_text)
        
        return chunks
    
    def _create_hierarchical_embeddings(self, chunks: List[LegalChunk]) -> List[LegalChunk]:
        """Tạo embedding phân cấp cho các chunk."""
        
        # Tổ chức chunks theo Điều
        article_dict = {}
        for chunk in chunks:
            article_id = chunk.metadata.article_id
            if article_id not in article_dict:
                article_dict[article_id] = []
            article_dict[article_id].append(chunk)
        
        # Tạo embedding cấp Điều
        article_embeddings = {}
        for article_id, article_chunks in article_dict.items():
            # Ghép tất cả chunk của điều này
            full_article_text = " ".join([chunk.text for chunk in article_chunks])
            
            # Tạo embedding cấp Điều
            article_embeddings[article_id] = self.model.encode(full_article_text)
        
        # Kết hợp embedding cấp Điều với embedding cấp chunk
        for chunk in chunks:
            # Tạo embedding cấp chunk
            chunk_embedding = self.model.encode(chunk.text)
            
            # Kết hợp với embedding cấp Điều (với trọng số)
            article_embedding = article_embeddings[chunk.metadata.article_id]
            combined_embedding = 0.7 * chunk_embedding + 0.3 * article_embedding
            
            # Chuẩn hóa
            norm = np.linalg.norm(combined_embedding)
            if norm > 0:
                combined_embedding = combined_embedding / norm
            
            chunk.embedding = combined_embedding
        
        return chunks

    def search(self, query: str, chunks: List[LegalChunk], top_k: int = 3) -> List[Dict]:
        """Tìm kiếm các chunk liên quan đến truy vấn."""
        query_embedding = self.model.encode(query)
        
        results = []
        for chunk in chunks:
            if chunk.embedding is not None:
                # Tính độ tương đồng cosine
                similarity = np.dot(query_embedding, chunk.embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(chunk.embedding)
                )
                
                results.append({
                    "chunk": chunk,
                    "similarity": float(similarity)
                })
        
        # Sắp xếp theo độ tương đồng
        results.sort(key=lambda x: x["similarity"], reverse=True)
        
        # Chuyển đổi kết quả sang định dạng dễ sử dụng
        formatted_results = []
        for result in results[:top_k]:
            chunk = result["chunk"]
            formatted_results.append({
                "text": chunk.text,
                "metadata": asdict(chunk.metadata),
                "similarity": result["similarity"]
            })
        
        return formatted_results

# Process Documents Using Custom Legal Processing

In [ ]:
# Process documents using the custom legal document processor
legal_processor = LegalDocument(max_chunk_tokens=500, overlap_tokens=100)

all_chunks = []
for doc in documents:
    # Xử lý mỗi tài liệu
    chunks = legal_processor.process(doc)
    all_chunks.extend(chunks)

print(f"Đã tạo {len(all_chunks)} chunks từ tài liệu.")

In [ ]:
# Create embeddings for all chunks
legal_embedder = LegalEmbedding(model_name='qducnguyen/vietnamese-bi-encoder')
chunks_with_embeddings = legal_embedder.create_embeddings(all_chunks, method='enhanced')

print(f"Đã tạo embedding cho {len(chunks_with_embeddings)} chunks.")

In [ ]:
# Convert to format compatible with llama_index
from llama_index.core import Document
from llama_index.core.schema import NodeRelationship, TextNode

# Create TextNodes from LegalChunks
nodes = []
for chunk in chunks_with_embeddings:
    # Create a TextNode
    node = TextNode(
        text=chunk.text,
        metadata=asdict(chunk.metadata),
        embedding=chunk.embedding
    )
    nodes.append(node)

print(f"Created {len(nodes)} TextNodes for indexing")
if len(nodes) > 0:
    print(f"Sample node metadata: {nodes[0].metadata}")

# Vietnamese Language Processing

In [ ]:
from pyvi import ViTokenizer
from tqdm import tqdm

# Tokenize text using ViTokenizer
for node in tqdm(nodes):
    # Save original text in metadata
    node.metadata["original_text"] = node.text
    # Tokenize and lowercase
    node.text = ViTokenizer.tokenize(node.text.lower())
    
    # Exclude original text from embedding
    if not hasattr(node, "excluded_embed_metadata_keys"):
        node.excluded_embed_metadata_keys = []
    if not hasattr(node, "excluded_llm_metadata_keys"):
        node.excluded_llm_metadata_keys = []
    
    node.excluded_embed_metadata_keys.append("original_text")
    node.excluded_llm_metadata_keys.append("original_text")

# Indexing

In [ ]:
# Save to disk
import weaviate
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from weaviate.classes.init import Auth


WEAVIATE_URL = "https://jd11sxlqap7tdknwzega.c0.asia-southeast1.gcp.weaviate.cloud"
weaviate_api_key = "93M51uT7bsG5EMnfL5z78woitWLg7XuAn4ps"
DATA_COLLECTION = "ND168_LEGAL_ENHANCED"
DEVICE = "cuda:0"

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

In [ ]:
# Configure vector store for pre-computed embeddings
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION,
    text_key="text",
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index using pre-computed embeddings
index = VectorStoreIndex(
    nodes, 
    storage_context=storage_context,
    embed_model=None,  # Use None since we already have embeddings
    insert_batch_size=32768,
    show_progress=True
)

# Test Retrieval

In [ ]:
from pyvi import ViTokenizer
from llama_index.core.response.notebook_utils import display_source_node

# Setup retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=10, 
    alpha=0.7
)

In [ ]:
# Test with our custom search function first
TEST_QUESTION = "đi xe máy không đội mũ bảo hiểm bị phạt bao nhiêu tiền?"
tokenized_query = ViTokenizer.tokenize(TEST_QUESTION.lower())

print("=== Tìm kiếm với LegalEmbedding.search ===\n")
custom_results = legal_embedder.search(tokenized_query, chunks_with_embeddings, top_k=5)

for i, result in enumerate(custom_results):
    print(f"\nKết quả #{i+1} (Độ tương đồng: {result['similarity']:.4f})")
    print(f"Điều {result['metadata']['article_id']}")
    print(f"Đoạn: {result['text'][:200]}...")

In [ ]:
# Test with llama_index retriever
print("=== Tìm kiếm với llama_index retriever ===\n")
retrievals = retriever.retrieve(tokenized_query)

for i, node in enumerate(retrievals[:5]):
    print(f"\nKết quả #{i+1} (Độ tương đồng: {node.score:.4f})")
    display_source_node(node, source_length=200, show_source_metadata=True)

# Compare Results

In [ ]:
# Comparison with other approaches (optional)
TEST_QUESTIONS = [
    "đi xe máy không đội mũ bảo hiểm bị phạt bao nhiêu tiền?",
    "phạt bao nhiêu cho người vượt đèn đỏ?",
    "điều khiển phương tiện mà trong máu có nồng độ cồn bị xử phạt thế nào?"
]

for question in TEST_QUESTIONS:
    tokenized_query = ViTokenizer.tokenize(question.lower())
    print(f"\n=== Câu hỏi: {question} ===\n")
    
    # Get results with llama_index retriever
    retrievals = retriever.retrieve(tokenized_query)
    
    # Display top result
    if retrievals:
        top_node = retrievals[0]
        print(f"Kết quả hàng đầu (Điểm: {top_node.score:.4f})")
        print(f"Điều: {top_node.metadata.get('article_id', 'N/A')}")
        print(f"Text: {top_node.text[:300]}...\n")